In [6]:
%load_ext rpy2.ipython
#https://gist.github.com/simecek/019d87c55fec3839d95bbf8489dde61d
from jupyter_pexpect_bash_console import pexpect_bash,pexpect_sudo_bash,pexpect_ssh_sudo_bash

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [7]:
%%R
R.version.string
print("email: 'Maciek Sykulski'<macieksk@gmail.com>")
length(ls())
getwd()

[1] "email: 'Maciek Sykulski'<macieksk@gmail.com>"
[1] "/home/hackyeah/hackyeah2018"


In [8]:
%%R
projDir<-"/home/hackyeah/hackyeah2018"
projectPrefix<-"R_airQuality"

#
load.sources<-function(){
    for (f in  c("utils.R" #Contains inpar, save_new_image, etc
                 #...  
                 ) 
        ){
    source(paste(projDir,"/src/",f,sep=""))
   }
}
load.sources()

In [9]:
%%R
#system.time(load(paste(projDir,"/r_saved_images/",projectPrefix,".Rdata",sep="")))
lsos()
gc()

          used (Mb) gc trigger (Mb) max used (Mb)
Ncells  554569 29.7     940480 50.3   750400 40.1
Vcells 1013386  7.8    1699674 13.0  1348735 10.3


In [10]:
%%R
gc()
#system.time(save_new_image())

          used (Mb) gc trigger (Mb) max used (Mb)
Ncells  554113 29.6     940480 50.3   750400 40.1
Vcells 1013011  7.8    1699674 13.0  1348735 10.3


In [12]:
%%R
library(lattice)
library(latticeExtra)
library(ggplot2)
library(gplots)
#library(png)

#library(Hmisc)
library(cluster)
library(MASS)
library(reshape)

#library(lumi)
#library(lumiHumanIDMapping)
#library(lumiHumanAll.db)
#library(limma)

#library(annotate)
#library(GOstats)

#library(multtest)
library(matrixStats)
#library(extrafont)
#library(enrichvs)
#library(statmod)
#library(flexmix)

load.sources()

sessionInfo()

R version 3.4.2 (2017-09-28)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux buster/sid

Matrix products: default
BLAS: /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
[1] C

attached base packages:
[1] parallel  tools     stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] matrixStats_0.52.2  reshape_0.8.7       MASS_7.3-47        
 [4] cluster_2.0.6       gplots_3.0.1        ggplot2_2.2.1      
 [7] latticeExtra_0.6-28 RColorBrewer_1.1-2  lattice_0.20-35    
[10] sqldf_0.4-11        RSQLite_2.0         gsubfn_0.6-6       
[13] proto_1.0.0         fastmatch_1.1-0     hash_2.2.6         

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.13       bit_1.1-12         munsell_0.4.3      colorspace_1.3-2  
 [5] rlang_0.1.4        plyr_1.8.4         blob_1.1.0         caTools_1.17.1    
 [9] tcltk_3.4.2        grid_3.4.2         gtable_0.2.0 

In [14]:
%%R 
#https://cran.r-project.org/mirrors.html

#install.packages(c("reshape"), #c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest'),
#                 repos="https://mirror.ibcp.fr/pub/CRAN/")

#repos='https://cran.us.r-project.org')
library(reshape)

In [15]:
import json
import codecs
import urllib.request as urllib2
import sys
from math import sin, cos, sqrt, atan2, radians
reader = codecs.getreader("utf-8")

allStationsUrl="http://api.gios.gov.pl/pjp-api/rest/station/findAll"
getIndexUrl="http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/"

def api(url,urlSuffix=""):
    data = json.load(reader(urllib2.urlopen("{0}{1}".format(url,urlSuffix))))
    return data

def getAllStationsAirQual():
    allStations=api(allStationsUrl)
    for station in allStations:
        airQual=api(getIndexUrl,station["id"])
        print("Stacja {}:\t{}".format(station["stationName"],airQual["stIndexLevel"]["indexLevelName"]))

def getDistance(x1,y1,x2,y2):
    R = 6373.0
    lat1 = radians(float(x1))
    lon1 = radians(float(y1))
    lat2 = radians(float(x2))
    lon2 = radians(float(y2))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R*c
    return distance

def getDistanceDict(userX,userY):
    allStations=api(allStationsUrl)
    distanceDict=dict()
    for station in allStations:
        distance=getDistance(userX,userY,station["gegrLat"],station["gegrLon"])
        if int(distance) not in distanceDict:
            distanceDict[int(distance)]=[]
        distanceDict[int(distance)].append([station["id"],station["stationName"]])
    return distanceDict

def getNearestIndexLevel(userX,userY):
    distanceDict=getDistanceDict(userX,userY)
    indexLevel="Brak indeksu"
    indexLevelId="Brak id"
    stationName="---"
    for dist in sorted(distanceDict.keys(),reverse=False):
        for station in distanceDict[dist]:
            obj=api(getIndexUrl, station[0])
            indexLevel=obj["stIndexLevel"]["indexLevelName"]
            indexLevelId=obj["stIndexLevel"]["id"]
            stationName=station[1]
            #print("Tmp:{3} {0} {1} {2}".format(stationName,indexLevel,indexLevelId,dist))
            if indexLevelId!=-1:
                return [stationName,indexLevel,indexLevelId]
    return [stationName,indexLevel,IndexLevelId]

In [16]:
print(getNearestIndexLevel(52.267316957465944,21.062164306640625)) #Warszawa
print(getNearestIndexLevel(50.04655739071663,19.934692382812504)) #Krakow
print(getNearestIndexLevel(52.40786463393834,16.938171386718754)) #Poznan

['Warszawa-Targówek', 'Umiarkowany', 2]
['Kraków, Aleja Krasińskiego', 'Bardzo zły', 5]
['Poznan-Dabrowskiego', 'Dostateczny', 3]
